In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.INFO)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from IPython.display import display

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    config_path = "./",
                    data_path = "../data/",
                    offline = True,
                    environment  = "local",
                    sources = ["acronet"]
                    )

INFO:ionbeam.sources.cima.cima:Looking for cache file at /Users/math/git/IonBeam_bundle/data/cache/acronet/cache.pickle
INFO:ionbeam.sources.cima.cima:Loaded cache file /Users/math/git/IonBeam_bundle/data/cache/acronet/cache.pickle


In [6]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

chains = [[s,] for s in sources]
for a in processors:
    if not isinstance(a.match, UUID): 
        chains.append([str(a.match), a,])
        continue
    for c in chains:
        matches = a.match.int == c[-1].id.int
        if matches: c.append(a)
            

for c in chains:
    print(" --> ".join(str(a) for a in c))

AcronetSource --> CSVParser --> AddAcronetMetadata --> ODCEncoder
[Match(state = 'odc_encoded')] --> FDBWriter()


In [7]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


id,3e929875-e71c-4fae-a0ca-574f97dbb9bd
mappings,"[InputColumn(name='time', key='time', type='datetime', unit=None, discard=False, canonical_variab..."
finish_after,None
copy_metadata_to_columns,"[station_id, station_name, lat, lon, start_time]"
cache_version,3
use_cache,True
source,acronet
cache_directory,/Users/math/git/IonBeam_bundle/data/cache/acronet
name,value
source_action_id,3e929875-e71c-4fae-a0ca-574f97dbb9bd
state,raw


Aggregators
Actions


id,e230b88b-d47e-4b71-915f-2da2412e0409
match,3e929875-e71c-4fae-a0ca-574f97dbb9bd
mappings,"[InputColumn(name='time', key='time', type='datetime', unit=None, discard=False, canonical_variab..."
identifying_keys,[time]
metadata_keys,"[author, station_id, lat, lon]"
separator,","
custom_nans,None
name,value
source_action_id,e230b88b-d47e-4b71-915f-2da2412e0409
state,parsed
mars_request,{}


id,4b6a12b3-278f-4a46-8e9b-860093891262
match,e230b88b-d47e-4b71-915f-2da2412e0409
name,value
source_action_id,4b6a12b3-278f-4a46-8e9b-860093891262
state,parsed
mars_request,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data
cache_path,/Users/math/git/IonBeam_bundle/data/cache
config_path,.


id,51786cca-ea2e-4c93-83ed-3f90579b8658
match,4b6a12b3-278f-4a46-8e9b-860093891262
output,outputs/{source}/odb/{external_id}_{start_time}.odb
MARS_keys,"[MARS_Key(name='class', dtype=<DataType.STRING: 3>, fill_method='constant', value='rd', key=None,..."
one_file_per_granule,True
split_data_columns,False
columns_to_metadata,[]
seconds,True
minutes,True
name,value
source_action_id,51786cca-ea2e-4c93-83ed-3f90579b8658


id,5ba99df2-5d77-4e4e-8bb1-b90ec8accd9e
match,[Match(state = 'odc_encoded')]
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_b..."
debug,[]
name,value
source_action_id,5ba99df2-5d77-4e4e-8bb1-b90ec8accd9e
state,written
mars_request,{}
name,value
canonical_variables,"[CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the obse..."
data_path,/Users/math/git/IonBeam_bundle/data


In [8]:
online = True
if online:
    source = sources[0]
    source.globals.offline = !online
    source.use_cache = True
    incoming_message_stream = source.generate()
    message = next(incoming_message_stream)
    message
else:
    p = Path('/Users/math/git/IonBeam-Deployment/data/inputs/meteotracker/MeteoTracker_62ae154f1d8e11061d4474b2.csv')
    message = FileMessage(
        metadata=MetaData(state='raw',
            source='meteotracker',
            filepath= p,
            variables = list(pd.read_csv(p).columns),             
            ))
message.metadata.unstructured

{'station': {'name': 'Monte Santa Croce',
  'id': 'monte_santa_croce',
  'lat': 44.148647,
  'lon': 9.679447,
  'sensors': {'PLUVIOMETRO': {'unit': 'mm', 'id': '-1937152391_2'},
   'TERMOMETRO': {'unit': '°C', 'id': '210331261_2'},
   'IGROMETRO': {'unit': '%', 'id': '210331262_2'},
   'DIREZIONEVENTO': {'unit': 'Degrees', 'id': '210331263_2'},
   'ANEMOMETRO': {'unit': 'm/s', 'id': '210331264_2'},
   'BAROMETRO': {'unit': 'hPa', 'id': '210331268_2'},
   'BATTERIA': {'unit': 'V', 'id': '210331258_2'},
   'TERMOMETRO_INTERNA': {'unit': '°C', 'id': '210331267_2'},
   'DIREZIONEVENTO_RAFFICA': {'unit': 'Degrees', 'id': '210331265_2'},
   'ANEMOMETRO_RAFFICA': {'unit': 'm/s', 'id': '210331266_2'},
   'SIGNAL_STRENGTH': {'unit': 'CSQ', 'id': '210331260_2'}}},
 'start': Timestamp('2024-12-10 06:35:00+0000', tz='UTC'),
 'end': Timestamp('2024-12-10 06:40:00+0000', tz='UTC')}

In [9]:
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator

possible_actions = [a for a in actions if not isinstance(a, Source)]

message_history = []
while True:
    display(message)
    message_history.append(message)
    matching = [action for action in possible_actions if action.matches(message)]
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))
        

TabularMessage(metadata=MetaData(source_action_id=UUID('3e929875-e71c-4fae-a0ca-574f97dbb9bd'), state='raw', source=None, observation_variable=None, time_span=TimeSpan(start=Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), end=Timestamp('2024-12-10 06:40:00+0000', tz='UTC')), encoded_format=None, filepath=None, mars_request={}, unstructured={'station': {'name': 'Monte Santa Croce', 'id': 'monte_santa_croce', 'lat': 44.148647, 'lon': 9.679447, 'sensors': {'PLUVIOMETRO': {'unit': 'mm', 'id': '-1937152391_2'}, 'TERMOMETRO': {'unit': '°C', 'id': '210331261_2'}, 'IGROMETRO': {'unit': '%', 'id': '210331262_2'}, 'DIREZIONEVENTO': {'unit': 'Degrees', 'id': '210331263_2'}, 'ANEMOMETRO': {'unit': 'm/s', 'id': '210331264_2'}, 'BAROMETRO': {'unit': 'hPa', 'id': '210331268_2'}, 'BATTERIA': {'unit': 'V', 'id': '210331258_2'}, 'TERMOMETRO_INTERNA': {'unit': '°C', 'id': '210331267_2'}, 'DIREZIONEVENTO_RAFFICA': {'unit': 'Degrees', 'id': '210331265_2'}, 'ANEMOMETRO_RAFFICA': {'unit': 'm/s', 'id': '210331266_2'}, 'SIGNAL_STRENGTH': {'unit': 'CSQ', 'id': '210331260_2'}}}, 'start': Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), 'end': Timestamp('2024-12-10 06:40:00+0000', tz='UTC')}), history=[], data=                       time  PLUVIOMETRO [mm]  TERMOMETRO [°C]  IGROMETRO [%]  \
0 2024-12-10 06:35:00+00:00               0.0              4.2           74.5   
1 2024-12-10 06:36:00+00:00               0.0              4.2           74.6   
2 2024-12-10 06:37:00+00:00               0.0              4.2           74.5   
3 2024-12-10 06:38:00+00:00               0.0              4.2           74.3   
4 2024-12-10 06:39:00+00:00               0.0              4.2           74.3   
5 2024-12-10 06:40:00+00:00               0.0              4.2           74.3   

   DIREZIONEVENTO [Degrees]  ANEMOMETRO [m/s]  BAROMETRO [hPa]  BATTERIA [V]  \
0                 285.60000               4.6        939.38000     12.500000   
1                 325.70000               3.7        939.39996     12.509999   
2                 246.90001               5.6        939.33997     12.490000   
3                 181.90001               5.8        939.36000     12.469999   
4                 281.40000               4.5        939.39996     12.480000   
5                 293.60000               4.2        939.42000     12.500000   

   TERMOMETRO_INTERNA [°C]  DIREZIONEVENTO_RAFFICA [Degrees]  \
0                      4.2                               NaN   
1                      4.2                               NaN   
2                      4.2                               NaN   
3                      4.2                               NaN   
4                      4.2                               NaN   
5                      4.2                               NaN   

   ANEMOMETRO_RAFFICA [m/s]  SIGNAL_STRENGTH [CSQ]   author  \
0                       7.3                    NaN  Acronet   
1                       4.9                    NaN  Acronet   
2                       6.9                    NaN  Acronet   
3                       7.0                   23.0  Acronet   
4                       6.1                    NaN  Acronet   
5                       5.8                    NaN  Acronet   

          station_id       station_name        lat       lon  \
0  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   
1  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   
2  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   
3  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   
4  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   
5  monte_santa_croce  Monte Santa Croce  44.148647  9.679447   

                 start_time  
0 2024-12-10 06:35:00+00:00  
1 2024-12-10 06:35:00+00:00  
2 2024-12-10 06:35:00+00:00  
3 2024-12-10 06:35:00+00:00  
4 2024-12-10 06:35:00+00:00  
5 2024-12-10 06:35:00+00:00  , columns=[])

That matched with:  ['CSVParser']


TabularMessage(metadata=MetaData(source_action_id=UUID('e230b88b-d47e-4b71-915f-2da2412e0409'), state='parsed', source=None, observation_variable='start_time,station_name,rain,air_temperature_near_surface,relative_humidity_near_surface,wind_direction_near_surface,wind_speed_near_surface,wind_speed_near_surface,wind_gust,air_pressure_near_surface,radiometer,battery_level,internal_temperature,wind_gust_direction,wind_gust,thermometer_min,thermometer_max,signal_strength', time_span=TimeSpan(start=Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), end=Timestamp('2024-12-10 06:40:00+0000', tz='UTC')), encoded_format=None, filepath=None, mars_request={}, unstructured={'station': {'name': 'Monte Santa Croce', 'id': 'monte_santa_croce', 'lat': 44.148647, 'lon': 9.679447, 'sensors': {'PLUVIOMETRO': {'unit': 'mm', 'id': '-1937152391_2'}, 'TERMOMETRO': {'unit': '°C', 'id': '210331261_2'}, 'IGROMETRO': {'unit': '%', 'id': '210331262_2'}, 'DIREZIONEVENTO': {'unit': 'Degrees', 'id': '210331263_2'}, 'ANEMOMETRO': {'unit': 'm/s', 'id': '210331264_2'}, 'BAROMETRO': {'unit': 'hPa', 'id': '210331268_2'}, 'BATTERIA': {'unit': 'V', 'id': '210331258_2'}, 'TERMOMETRO_INTERNA': {'unit': '°C', 'id': '210331267_2'}, 'DIREZIONEVENTO_RAFFICA': {'unit': 'Degrees', 'id': '210331265_2'}, 'ANEMOMETRO_RAFFICA': {'unit': 'm/s', 'id': '210331266_2'}, 'SIGNAL_STRENGTH': {'unit': 'CSQ', 'id': '210331260_2'}}}, 'start': Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), 'end': Timestamp('2024-12-10 06:40:00+0000', tz='UTC')}), history=[PreviousActionInfo(action=ActionInfo(name='CSVParser', code=CodeSourceInfo(repo_status='Dirty', git_hash='4220093fddf63ffb45633d9f392b2813b3f7e55c')), message=MessageInfo(name='TabularMessage', metadata=MetaData(source_action_id=UUID('3e929875-e71c-4fae-a0ca-574f97dbb9bd'), state='raw', source=None, observation_variable=None, time_span=TimeSpan(start=Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), end=Timestamp('2024-12-10 06:40:00+0000', tz='UTC')), encoded_format=None, filepath=None, mars_request={}, unstructured={'station': {'name': 'Monte Santa Croce', 'id': 'monte_santa_croce', 'lat': 44.148647, 'lon': 9.679447, 'sensors': {'PLUVIOMETRO': {'unit': 'mm', 'id': '-1937152391_2'}, 'TERMOMETRO': {'unit': '°C', 'id': '210331261_2'}, 'IGROMETRO': {'unit': '%', 'id': '210331262_2'}, 'DIREZIONEVENTO': {'unit': 'Degrees', 'id': '210331263_2'}, 'ANEMOMETRO': {'unit': 'm/s', 'id': '210331264_2'}, 'BAROMETRO': {'unit': 'hPa', 'id': '210331268_2'}, 'BATTERIA': {'unit': 'V', 'id': '210331258_2'}, 'TERMOMETRO_INTERNA': {'unit': '°C', 'id': '210331267_2'}, 'DIREZIONEVENTO_RAFFICA': {'unit': 'Degrees', 'id': '210331265_2'}, 'ANEMOMETRO_RAFFICA': {'unit': 'm/s', 'id': '210331266_2'}, 'SIGNAL_STRENGTH': {'unit': 'CSQ', 'id': '210331260_2'}}}, 'start': Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), 'end': Timestamp('2024-12-10 06:40:00+0000', tz='UTC')})))], data=                       time  rain  air_temperature_near_surface  \
0 2024-12-10 06:35:00+00:00   0.0                        277.35   
1 2024-12-10 06:36:00+00:00   0.0                        277.35   
2 2024-12-10 06:37:00+00:00   0.0                        277.35   
3 2024-12-10 06:38:00+00:00   0.0                        277.35   
4 2024-12-10 06:39:00+00:00   0.0                        277.35   
5 2024-12-10 06:40:00+00:00   0.0                        277.35   

   relative_humidity_near_surface  wind_direction_near_surface  \
0                            74.5                    285.60000   
1                            74.6                    325.70000   
2                            74.5                    246.90001   
3                            74.3                    181.90001   
4                            74.3                    281.40000   
5                            74.3                    293.60000   

   wind_speed_near_surface  air_pressure_near_surface  battery_level  \
0                 2.366442                  93938.000      12.500000   
1                 1.903443    

That matched with:  ['AddAcronetMetadata']


name,value
source_action_id,4b6a12b3-278f-4a46-8e9b-860093891262
state,parsed
observation_variable,"start_time,station_name,rain,air_temperature_near_surface,relative_humidity_near_surface,wind_dir..."
time_span,"TimeSpan(start=Timestamp('2024-12-10 06:35:00+0000', tz='UTC'), end=Timestamp('2024-12-10 06:40:0..."
mars_request,{}
name,value
name,TabularMessage
metadata,"MetaData(source = None, variable = None)"
name,value
name,CSVParser


That matched with:  ['ODCEncoder']


RuntimeError: Unknown data type datetime64[ns, UTC] not supported

In [ ]:
m = message_history[-1]
m.metadata

In [ ]:
for i, message in enumerate(message_history):
    print(i, message.metadata.time_span)

In [ ]:

df = pd.DataFrame({"class" : ["rd", "rd"], "col1" : [1,2], "col2" : [1,2]})
pyodc.encode_odb(df, "test.odb")

with open("test.odb", "rb") as f:
    pyfdb.archive(f.read())

In [ ]:
def round_datetime(dt: datetime, round_to: int = 5, method: str = "floor") -> datetime:
    if round_to <= 0:
        raise ValueError("round_to must be a positive integer")
    if method not in {"floor", "ceil"}:
        raise ValueError("method must be 'floor' or 'ceil'")
    
    # Calculate the number of seconds since the start of the day
    total_seconds = (dt - dt.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    
    # Calculate the rounded total seconds
    rounding_seconds = round_to * 60
    if method == "floor":
        rounded_seconds = (total_seconds // rounding_seconds) * rounding_seconds
    else:  # method == "ceil"
        rounded_seconds = ((total_seconds + rounding_seconds - 1) // rounding_seconds) * rounding_seconds
    
    # Return the rounded datetime
    return dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=rounded_seconds)

# Example usage:
dt = datetime.now()
print("Original:", dt)
print("Rounded to floor:", round_datetime(dt, round_to=5, method="floor"))
print("Rounded to ceil:", round_datetime(dt, round_to=5, method="ceil"))